In [1]:
import dask
import os
import re
import datetime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import importlib

from letkf_forecasting import analyse_results, letkf_io

In [22]:
importlib.reload(analyse_results)
importlib.reload(letkf_io)

base_folder = '/a2/uaren/travis'

decimals = 2

year = 2014

# month = 4
# day = 15
# day_name = 'translation'

# month = 5
# day = 29
# day_name = 'more_complex'

month = 4
day = 26
day_name = 'two_levels'

dates = [datetime.date(2014, month, day)]
runs = ['owp_opt', 'persistence', 'opt_flow', 'wrf_no_div', 'wrf_mean', 'radiosonde']
# runs = [ 'owp_opt', 'persistence', 'opt_flow', 'wrf_no_div',
#         'opt_flow_with_div', 'wrf', 'radiosonde', 'wrf_mean']
# runs = ['wrf_mean']

In [23]:
save_directory = "/home2/travis/python_code/letkf_forecasting/tables/"

In [24]:
horizons = [15, 30, 45, 60]

In [25]:
legend_dict = {'opt_flow': 'Opt.~Flow', 
               'opt_flow_with_div': 'Opt. Flow w/ Div.',
               'wrf_no_div': 'NWP Winds',
               'wrf': 'NWP w/ Div.',
               'owp_opt': 'ANOC',                                                                                
               'persistence': 'Persis.',
               'radiosonde': 'Radiosonde',
               'wrf_mean': 'NWP Avg.~Winds',
               'ens_member': 'Ens.~Member'} 

In [26]:
rmse = pd.DataFrame(index=horizons, columns=runs)
rmse.index.name = 'Horizon'
correlation = rmse.copy()
bias = rmse.copy()
for run_name in runs:
    results_folder_path = os.path.join(                                          
        base_folder,                                                               
        'results',                                                               
        f'{year:04}',                                                            
        f'{month:02}',                                                           
        f'{day:02}',                                                             
        run_name)
    results_folder_path = letkf_io.find_latest_run(results_folder_path)
    results_folder_path = os.path.join(results_folder_path, 'single_day')
    
    stat_name = 'rmse'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    rmse[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'bias'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    bias[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'correlation'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    correlation[run_name] = pd.read_hdf(file_path, stat_name)

In [27]:
peices = [rmse, correlation, bias]
combined = pd.concat(peices, axis=0,
                     keys=['RMSE', 'Corr.', 'Bias'])
combined = combined.rename(columns=legend_dict)

In [28]:
def is_empty(str):
    return str != ''

In [29]:
def format_table(text, header_num=5, footer_num=2):
    text = text.split(' ')
    text = list(filter(is_empty, text))
    text = ' '.join(text)
    split_text = text.split('\n')
    split_titles2 = split_text[2]
    removed = split_titles2[-2:]
    split_titles2 = split_titles2[:-2]
    split_titles2 = split_titles2.split('&')
    for count, this in enumerate(split_titles2):
        if len(this) > 2:
            this = this[0] + '{' + this[1:-1] + '}' + this[-1]
            split_titles2[count] = this
    split_text[2] = '&'.join(split_titles2) + removed
#     split_titles2 = split_text[2].split('&')
#     split_titles3 = split_text[3].split('&')
#     for num, title in enumerate(split_text[2].split('&')):
#         if ' w/ Div.' in title:
#             split_titles2[num] = title[:-8]
#             split_titles3[num] = title[-9:]
#     split_titles2 = '&'.join(split_titles2)
#     split_titles3 = '&'.join(split_titles3)
#     split_text[2] = split_titles2
#     split_text[3] = split_titles3
    
    for line_num, line in enumerate(split_text[header_num:-footer_num - 1]):
        split_line = line.split(' ')
        if split_line[0] == 'Corr.':
            Corr = True
        elif split_line[0] != '':
            Corr = False
        num_slice = slice(4, None, 2)
        numbers_str = split_line[num_slice]
        numbers = np.array(
            split_line[num_slice],
            dtype='float')
        if Corr:
            best_num = numbers.max()
        else:
            best_num = numbers[np.abs(numbers).argmin()]
        argmins = np.where(numbers == best_num)[0]
#         numbers = list(numbers.astype('str'))
        for argmin in argmins:
            numbers_str[argmin] = '\\B ' + numbers_str[argmin]
#             numbers_str[argmin] = '\\textbf{' + numbers_str[argmin] + '}'
        split_line[num_slice] = numbers_str
        split_text[header_num + line_num] = ' '.join(split_line)
    
#     for count in range(hor_num - 1):
#         split_line.insert(((count + 1)*run_num)*2 + 2 + count, '&')
#     split_line

    
    return '\n'.join(split_text)

In [30]:
column_format = 'll' + 'S[table-format=-1.3]' * len(runs)
text = combined.round(decimals=decimals).to_latex(column_format=column_format)
text2 = format_table(text)
text2 = re.sub('\\\\textasciitilde', '~', text2, count=5)
print(text2)

\begin{tabular}{llS[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]S[table-format=-1.3]}
\toprule
 & & {ANOC} & {Persis.} & {Opt.~Flow} & {NWP Winds} & {NWP Avg.~Winds} & {Radiosonde} \\
{} & Horizon & & & & & & \\
\midrule
RMSE & 15 & \B 0.23 & 0.24 & \B 0.23 & 0.30 & 0.29 & 0.36 \\
 & 30 & \B 0.28 & 0.32 & 0.29 & 0.40 & 0.38 & 0.39 \\
 & 45 & \B 0.29 & 0.37 & 0.34 & 0.40 & 0.41 & 0.37 \\
 & 60 & \B 0.29 & 0.38 & 0.35 & 0.39 & 0.39 & 0.35 \\
Corr. & 15 & \B 0.76 & 0.73 & 0.75 & 0.61 & 0.63 & 0.40 \\
 & 30 & \B 0.63 & 0.52 & 0.62 & 0.24 & 0.32 & 0.24 \\
 & 45 & \B 0.60 & 0.36 & 0.48 & 0.18 & 0.16 & 0.28 \\
 & 60 & \B 0.60 & 0.35 & 0.49 & 0.23 & 0.27 & 0.40 \\
Bias & 15 & -0.02 & \B 0.01 & 0.02 & -0.10 & -0.08 & -0.10 \\
 & 30 & -0.03 & \B 0.01 & 0.04 & -0.12 & -0.10 & -0.04 \\
 & 45 & -0.04 & \B 0.00 & 0.04 & -0.11 & -0.10 & 0.01 \\
 & 60 & -0.07 & -0.03 & 0.02 & -0.09 & -0.07 & \B -0.01 \\
\bottomrule
\end{tabular}



In [31]:
this_file = os.path.join(save_directory, f'{day_name}_results.tex')
with open(this_file, 'w') as file:
    file.write(text2)